## Exploit Results
Retrieve job output files linked from the job status report

In [ ]:
from urllib.parse import urlparse
import requests
import json
from pystac import STAC_IO
from requests.auth import HTTPBasicAuth
from pystac import *
import matplotlib.pyplot as plt
import getpass 
import gdal

In [ ]:
# Job already completed
# ---------------------
job_location = "/watchjob/processes/vegetation_index_/jobs/f99f877e-1941-11eb-b04f-0242ac110006"
# ---------------------
endpoint = 'http://proc-ades.test.192.168.49.2.nip.io'
token = "eyJhbGciOiJIUzI1NiIsImtpZCI6IlJTQTEifQ.eyJhY3RpdmUiOnRydWUsImV4cCI6MTU5MzUxNTU2NSwiaWF0IjoxNTkzNTExOTY1LCJuYmYiOm51bGwsInBlcm1pc3Npb25zIjpbeyJyZXNvdXJjZV9pZCI6ImI3Y2FkZTVjLTM3MmYtNGM4Ny1iZTgyLWE3OTU2NDk4ZTcyOSIsInJlc291cmNlX3Njb3BlcyI6WyJBdXRoZW50aWNhdGVkIiwib3BlbmlkIl0sImV4cCI6MTU5MzUxNTU2NCwicGFyYW1zIjpudWxsfV0sImNsaWVudF9pZCI6IjYxY2UyOGQ1LWFhMTYtNGRkYy04NDJmLWZjYzE1OGQzMTVmYSIsInN1YiI6bnVsbCwiYXVkIjoiNjFjZTI4ZDUtYWExNi00ZGRjLTg0MmYtZmNjMTU4ZDMxNWZhIiwiaXNzIjpudWxsLCJqdGkiOm51bGwsInBjdF9jbGFpbXMiOnsiYXVkIjpbIjYxY2UyOGQ1LWFhMTYtNGRkYy04NDJmLWZjYzE1OGQzMTVmYSJdLCJzdWIiOlsiZWIzMTQyMWUtMGEyZS00OTBmLWJiYWYtMDk3MWE0ZTliNzhhIl0sInVzZXJfbmFtZSI6WyJyaWNvbndheSJdLCJpc3MiOlsiaHR0cHM6Ly9lb2VwY2EtZGV2LmRlaW1vcy1zcGFjZS5jb20iXSwiZXhwIjpbIjE1OTM1MTU1NjQiXSwiaWF0IjpbIjE1OTM1MTE5NjQiXSwib3hPcGVuSURDb25uZWN0VmVyc2lvbiI6WyJvcGVuaWRjb25uZWN0LTEuMCJdfX0.d5qeaqLfl0oh9KigVrM_lT1hZMaOzQBFB7jjaKI3PjE"
headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json', 'Accept': 'application/json'}

### Job Status

In [ ]:
r = requests.get(f'{endpoint}{job_location}', headers=headers)
print(r.status_code, r.reason)
response = r.json()
print(json.dumps(response, indent=2))

### Job Results - (GET /processes/{id}/jobs/{job_id}/result)

In [ ]:
# Request the results
# -------------------
r = requests.get(f'{endpoint}/{job_location}/result', headers=headers)
# -------------------
print(r.status_code, r.reason)

# Interpret the response
response = r.json()
results = json.loads(response['outputs'][0]['value']['inlineValue'])
print(json.dumps(results, indent=2))
stac_catalog_endpoint = results['stac:catalog']['href']
print(f"STAC catalogue results index = {stac_catalog_endpoint}")

## Exploiting the results

Discovery and exploition of the generated results using STAC  

### Enumerate the result files from the STAC catalogue file

In [ ]:
%matplotlib inline
# Credentials for 'workspace' to access results
username = 'eoepca'
password = "telespazio"

# Configure STAC library to access workspace via http
def my_read_method(uri):
    parsed = urlparse(uri)
    if parsed.scheme.startswith('http'):
        return requests.get(uri, auth=HTTPBasicAuth(username, password)).text
    else:
        return STAC_IO.default_read_text_method(uri)
STAC_IO.read_text_method = my_read_method

# Interrogate the STAC catalgue file to enumerate the results
catalog = Catalog.from_file(stac_catalog_endpoint)
item = next(catalog.get_items())
print(f"result assets =\n{item.get_assets()}")
nbr_asset = item.get_assets()['nbr']
nbr_asset.get_absolute_href()
print(f"Normalized burn ratio =\n{nbr_asset.get_absolute_href()}")

### Visualise the Normalized Burn Ratio

In [ ]:
# GDAL url prep
def get_vsi_url(uri, username=None, password=None):
    parsed_url = urlparse(uri)
    if username is not None:
        url = '/vsicurl/{}://{}:{}@{}/{}'.format(list(parsed_url)[0], username, password, list(parsed_url)[1], list(parsed_url)[2])
    else:
        url = '/vsicurl/{}://{}/{}'.format(list(parsed_url)[0], list(parsed_url)[1], list(parsed_url)[2])
    return url

# Retrieve the data as numpy array
ds = gdal.Open(get_vsi_url(uri=nbr_asset.get_absolute_href(), username=username,  password=password))
nbr_band = ds.GetRasterBand(1)
nbr_data = nbr_band.ReadAsArray()

# Visualise
fig = plt.figure(figsize=(25,25))
imgplot = plt.imshow(nbr_data.reshape(nbr_data.shape[0], nbr_data.shape[1]), cmap=plt.cm.gray)